# 건물 데이터 EDA 및 전처리

* 목표: 학습데이터 구축을 위해 각각의 데이터 구조를 파악하고 적합한 형태로 가공한 후 재저장

In [1]:
# import libraries

import sys
import os
import pathlib
from pathlib import Path
import datetime
from datetime import datetime
import time
import warnings
warnings.filterwarnings(action='ignore')
from tqdm import tqdm

import pandas as pd
import numpy as np
import json
import folium
import geopandas as gpd
import fiona
from shapely.geometry import Point, LineString
import pyproj

import matplotlib as mat
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from matplotlib import font_manager, rc

plt.rcParams['figure.figsize'] = (20,10)
mat.rcParams['axes.unicode_minus'] = False

import io
from PIL import Image

from geoband import API
from geoband.API import *

print(sys.version)

3.6.9 (default, Nov 11 2019, 11:24:16) 
[GCC 4.8.5 20150623 (Red Hat 4.8.5-39)]


In [2]:
input_path = pathlib.Path('./input')
if not input_path.is_dir():
    input_path.mkdir()

# 대회데이터

bd = pd.read_csv(input_path.joinpath('4.부산시남구_건축물대장(표제부).csv'))
addr_bd = gpd.read_file(input_path.joinpath('5.부산시남구_도로명주소(건물).geojson'))
age = gpd.read_file(input_path.joinpath('6.부산시남구_주택노후도.geojson'))
prc = gpd.read_file(input_path.joinpath('10.부산시남구_공시지가.geojson'))

---

## 대회데이터

### 건축물대장(표제부)

* 현재 기준의 부산시 남구 건축물대장 표제부 데이터(건물 노후도(사용승인일자) 및 주차 용량 정보 활용 데이터)

* 학습데이터의 기본 틀
  
  - 이 데이터프레임을 바탕으로 결측값을 채우고, 새로운 컬럼을 추가할 예정

In [3]:
bd.head(1)

,loc,sgg_cd,emd_cd,land_gbn,bun,ji,reg_cd,reg_fld_cd,address,emd_nm,...,in_mec_area,out_mec_num,out_mec_area,in_sfp_num,in_sfp_area,out_sfp_num,out_sfp_area,us_athr_date,lon,lat
0,부산광역시 남구 대연동 1-4번지,26290,10600,0,"""0001""","""0004""",일반,일반건축물,,NaN,...,0.0,0,0.0,0,0.0,0,0.0,19750211.0,129.101522,35.136334


In [4]:
bd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26101 entries, 0 to 26100
Data columns (total 28 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   loc           26101 non-null  object 
 1   sgg_cd        26101 non-null  int64  
 2   emd_cd        26101 non-null  int64  
 3   land_gbn      26101 non-null  int64  
 4   bun           26101 non-null  object 
 5   ji            26101 non-null  object 
 6   reg_cd        26101 non-null  object 
 7   reg_fld_cd    26101 non-null  object 
 8   address       26101 non-null  object 
 9   emd_nm        3337 non-null   object 
 10  mbd_sep_cd    26100 non-null  object 
 11  mus_cd_nm     26093 non-null  object 
 12  etc_us_info   26085 non-null  object 
 13  house_num     26101 non-null  int64  
 14  fam_num       26101 non-null  int64  
 15  grd_flr_num   26101 non-null  int64  
 16  ugr_flr_num   26101 non-null  int64  
 17  in_mec_num    26101 non-null  int64  
 18  in_mec_area   26101 non-nu

#### 컬럼별 설명

* loc: 대지 위치 정보
  - 번지 주소
* sgg_cd: 시군구 코드
  - 부산광역시 코드
* emd_cd: 법정동 코드
* land_gbn: 대지구분코드
  - 0: 일반, 1: 산, 2: 가번지
* bun: 번 정보
* ji: 지 정보
* reg_cd: 대장의 구분코드명
* reg_fld_cd: 대장의 종류코드명
* address: 대지의 도로명 주소
* emd_nm: 동명칭
* mbd_sep_cd: 주부속 구분 코드 정보
* mus_cd_nm: 주용도 코드 이름
* etc_us_info: 기타 용도 정보
* house_num: 세대 수
* fam_num: 가구 수
* grd_flr_num: 지상 층 수
* ugr_flr_num: 지하 층 수
* in_mec_num: 옥내기계식 대수
* in_mec_area: 옥내기계식 면적
* out_mec_num: 옥외 기계식 대수
* out_mec_area: 옥외 기계식 면적
* in_sfp_num: 옥내 자주식 대수
* in_sfp_area: 옥내 자주식 면적
* out_sfp_num: 옥외 자주식 대수
* out_sfp_area: 옥외 자주식 면적
* us_athr_date: 사용승인 날짜
* lon: 경도
* lat: 위도

In [5]:
# 컬럼별 결측값 갯수 확인

bd.isnull().sum()

loc                 0
sgg_cd              0
emd_cd              0
land_gbn            0
bun                 0
ji                  0
reg_cd              0
reg_fld_cd          0
address             0
emd_nm          22764
mbd_sep_cd          1
mus_cd_nm           8
etc_us_info        16
house_num           0
fam_num             0
grd_flr_num         0
ugr_flr_num         0
in_mec_num          0
in_mec_area         0
out_mec_num         0
out_mec_area        0
in_sfp_num          0
in_sfp_area         0
out_sfp_num         0
out_sfp_area        0
us_athr_date     4344
lon                 0
lat                 0
dtype: int64

In [6]:
bd['sgg_cd'].unique() #  제거 (새로운 컬럼 생성)

array([26290])

In [7]:
bd['emd_cd'].value_counts() # 제거 (새로운 컬럼 생성)

10600    8569
10900    6352
10700    4594
11100    3362
11000    2121
10800    1103
Name: emd_cd, dtype: int64

In [8]:
bd['land_gbn'].value_counts()

0    25643
1      444
2       14
Name: land_gbn, dtype: int64

In [9]:
bd['reg_cd'].unique()

array(['일반', '집합'], dtype=object)

In [10]:
bd['reg_fld_cd'].unique()

array(['일반건축물', '표제부'], dtype=object)

In [11]:
bd['address'].nunique()

18532

In [12]:
print(bd['etc_us_info'].nunique(), bd['etc_us_info'].unique())

1434 ['주택' '근린생활시설' ' 근린생활시설, 교육연구시설, 업무시설 ' ... '주택  및 점포' '대웅전' '산신각']


In [13]:
bd['mbd_sep_cd'].unique()

array(['주건축물', '부속건축물', nan], dtype=object)

In [14]:
bd['mus_cd_nm'].unique()

array(['단독주택', '제2종근린생활시설', '공동주택', '제1종근린생활시설', '위험물저장및처리시설', '업무시설',
       '자동차관련시설', '노유자시설', '교육연구시설', '종교시설', '공장', '의료시설', '창고시설', '숙박시설',
       '동.식물관련시설', nan, '근린생활시설', '방송통신시설', '위락시설', '판매시설', '운수시설',
       '운동시설', '문화및집회시설', '교육연구및복지시설', '관광휴게시설', '자원순환관련시설', '분뇨.쓰레기처리시설',
       '교정및군사시설'], dtype=object)

In [15]:
# 숫자형 컬럼 mean, max, min 등 확인

pd.options.display.float_format = '{:.3f}'.format # 소수점 이하 셋째 자리까지만 표시

num_cols = ['house_num','fam_num','grd_flr_num','ugr_flr_num','in_mec_num',
            'in_mec_area','out_mec_num','out_mec_area','in_sfp_num','in_sfp_area','out_sfp_num','out_sfp_area']

bd[num_cols].describe()

,house_num,fam_num,grd_flr_num,ugr_flr_num,in_mec_num,in_mec_area,out_mec_num,out_mec_area,in_sfp_num,in_sfp_area,out_sfp_num,out_sfp_area
count,26101.000,26101.000,26101.000,26101.000,26101.000,26101.000,26101.000,26101.000,26101.000,26101.000,26101.000,26101.000
mean,3.620,1.130,2.532,0.192,0.271,1.068,0.019,0.057,11.955,94.793,3.726,32.602
std,18.683,3.822,3.500,0.465,4.303,20.016,0.728,2.048,219.325,1441.106,44.377,502.757
min,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
50%,0.000,1.000,2.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
75%,0.000,1.000,3.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
max,521.000,400.000,69.000,12.000,238.000,1493.290,80.000,230.000,5399.000,45208.670,1708.000,44875.000


In [16]:
# 새로운 컬럼 (emd_jibun) 만들기

ji = bd['ji'].to_list()
ji_2 = [x.replace('"','') for x in ji]
ji_3 = list(map(int, ji_2))
ji_4 = list(map(str, ji_3))

bun = bd['bun'].to_list()
bun_2 = [y.replace('"','') for y in bun]
bun_3 = list(map(int, bun_2))
bun_4 = list(map(str, bun_3))

bd['new_ji'] = ji_4
bd['new_bun'] = bun_4

bd['emd_jibun'] = bd['sgg_cd'].map(str) + bd['emd_cd'].map(str) + '(' + bd['new_ji'] + '-' + bd['new_bun'] + ')'

bd.head(1)

,loc,sgg_cd,emd_cd,land_gbn,bun,ji,reg_cd,reg_fld_cd,address,emd_nm,...,in_sfp_num,in_sfp_area,out_sfp_num,out_sfp_area,us_athr_date,lon,lat,new_ji,new_bun,emd_jibun
0,부산광역시 남구 대연동 1-4번지,26290,10600,0,"""0001""","""0004""",일반,일반건축물,,NaN,...,0,0.000,0,0.000,19750211.000,129.102,35.136,4,1,2629010600(4-1)


In [17]:
# 불필요한 컬럼 제거

bd_temp = bd.drop(columns = ['loc','sgg_cd','emd_cd','bun','ji','new_ji','new_bun'], axis=1)

bd_temp.head(1)

,land_gbn,reg_cd,reg_fld_cd,address,emd_nm,mbd_sep_cd,mus_cd_nm,etc_us_info,house_num,fam_num,...,out_mec_num,out_mec_area,in_sfp_num,in_sfp_area,out_sfp_num,out_sfp_area,us_athr_date,lon,lat,emd_jibun
0,0,일반,일반건축물,,NaN,주건축물,단독주택,주택,0,1,...,0,0.000,0,0.000,0,0.000,19750211.000,129.102,35.136,2629010600(4-1)


In [18]:
# us_athr_date의 데이터 타입 변경

bd_temp['us_athr_date'] = pd.to_datetime(bd_temp['us_athr_date'], format='%Y%m%d', errors='coerce')
print(bd_temp.dtypes)
bd_temp.head(1)

land_gbn                 int64
reg_cd                  object
reg_fld_cd              object
address                 object
emd_nm                  object
mbd_sep_cd              object
mus_cd_nm               object
etc_us_info             object
house_num                int64
fam_num                  int64
grd_flr_num              int64
ugr_flr_num              int64
in_mec_num               int64
in_mec_area            float64
out_mec_num              int64
out_mec_area           float64
in_sfp_num               int64
in_sfp_area            float64
out_sfp_num              int64
out_sfp_area           float64
us_athr_date    datetime64[ns]
lon                    float64
lat                    float64
emd_jibun               object
dtype: object


,land_gbn,reg_cd,reg_fld_cd,address,emd_nm,mbd_sep_cd,mus_cd_nm,etc_us_info,house_num,fam_num,...,out_mec_num,out_mec_area,in_sfp_num,in_sfp_area,out_sfp_num,out_sfp_area,us_athr_date,lon,lat,emd_jibun
0,0,일반,일반건축물,,NaN,주건축물,단독주택,주택,0,1,...,0,0.000,0,0.000,0,0.000,1975-02-11,129.102,35.136,2629010600(4-1)


In [19]:
# 컬럼 재배치

col_1 = bd_temp.columns[-1:].to_list()
col_2 = bd_temp.columns[:-1].to_list()
new_col = col_1 + col_2
cp_bd = bd_temp[new_col]
cp_bd.head(1) # competition building

,emd_jibun,land_gbn,reg_cd,reg_fld_cd,address,emd_nm,mbd_sep_cd,mus_cd_nm,etc_us_info,house_num,...,in_mec_area,out_mec_num,out_mec_area,in_sfp_num,in_sfp_area,out_sfp_num,out_sfp_area,us_athr_date,lon,lat
0,2629010600(4-1),0,일반,일반건축물,,NaN,주건축물,단독주택,주택,0,...,0.000,0,0.000,0,0.000,0,0.000,1975-02-11,129.102,35.136


In [20]:
cp_bd.to_csv(input_path.joinpath('대회_건축물대장_정리.csv'), index_label=False)

---

## 건축물연령

* 현재 기준의 부산시 남구 주택노후도 데이터

In [21]:
age.head(2)

,emd_cd,emd_nm,spc_grd_cd,spc_grd_info,jibun,co_bd_cd,co_bd_nm,bd_nm,dong,bd_area,mus_cd,mus_cd_nm,bd_h,grd_flr_num,ugr_flr_num,athr_date,us_athr_date,bd_age,geometry
0,2629010600,부산광역시 남구 대연동,1,일반,6-5,2,집합건축물,레이듐오피스텔,None,1830.380,14000,업무시설,29.400,9,1,2001-04-10,2002-07-26,20.000,"MULTIPOLYGON (((129.10670 35.13866, 129.10663 ..."
1,2629010600,부산광역시 남구 대연동,1,일반,6-8,2,집합건축물,대박세븐,None,919.730,02000,공동주택,17.400,6,0,2012-03-13,2012-07-26,10.000,"MULTIPOLYGON (((129.10617 35.13848, 129.10616 ..."


In [22]:
age.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 20534 entries, 0 to 20533
Data columns (total 19 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   emd_cd        20534 non-null  object  
 1   emd_nm        20534 non-null  object  
 2   spc_grd_cd    20534 non-null  object  
 3   spc_grd_info  20534 non-null  object  
 4   jibun         20534 non-null  object  
 5   co_bd_cd      20534 non-null  object  
 6   co_bd_nm      20534 non-null  object  
 7   bd_nm         7177 non-null   object  
 8   dong          2429 non-null   object  
 9   bd_area       20534 non-null  float64 
 10  mus_cd        20529 non-null  object  
 11  mus_cd_nm     20524 non-null  object  
 12  bd_h          20534 non-null  float64 
 13  grd_flr_num   20534 non-null  int64   
 14  ugr_flr_num   20534 non-null  int64   
 15  athr_date     15347 non-null  object  
 16  us_athr_date  18755 non-null  object  
 17  bd_age        18755 non-null  float64 
 18

#### 컬럼별 설명

* emd_cd:    법정동 코드
  - 10600 = 대연동, 10900 = 문현동, 10700 = 용호동, 10800 = 용당동, 11000 = 우암동, 11100 = 감만동
* emd_nm:    법정동 이름
* spc_grd_cd:    특수지 구분 코드
  - 1: 일반, 2: 산
* spc_grd_info:    특수지 구분 정보
* jibun:    지번
* co_bd_cd:    집합건물구분코드
  - 1: 일반건축물, 2: 집합건축물
* co_bd_nm:    집합건물구분명
* bd_nm:    건물명
* dong:    동명
* bd_area:    건물연면적
* mus_cd:    주요 용도 코드
* mus_cd_nm:    주요 용도 이름
* bd_h:    건물높이
* grd_flr_num:    지상층수
* ugr_flr_num:    지하 층 수
* athr_date:    허가 일자 (착공)
* us_athr_date:    사용승인 일자 (준공 시점 --- 노후도 산출 기준)
* bd_age:    건물의 연령 (20-30: 노후건축물기준) (공공데이터로 null 값 대체)

In [23]:
age.isnull().sum()

emd_cd              0
emd_nm              0
spc_grd_cd          0
spc_grd_info        0
jibun               0
co_bd_cd            0
co_bd_nm            0
bd_nm           13357
dong            18105
bd_area             0
mus_cd              5
mus_cd_nm          10
bd_h                0
grd_flr_num         0
ugr_flr_num         0
athr_date        5187
us_athr_date     1779
bd_age           1779
geometry            0
dtype: int64

In [24]:
age['emd_cd'].unique() # 10600 = 대연동, 10900 = 문현동, 10700 = 용호동, 10800 = 용당동, 11000 = 우암동, 11100 = 감만동

array(['2629010600', '2629010700', '2629010800', '2629010900',
       '2629011000', '2629011100'], dtype=object)

In [25]:
age['emd_nm'].unique()

array(['부산광역시 남구 대연동', '부산광역시 남구 용호동', '부산광역시 남구 용당동', '부산광역시 남구 문현동',
       '부산광역시 남구 우암동', '부산광역시 남구 감만동'], dtype=object)

In [26]:
age['spc_grd_cd'].unique() # 1 = 일반, 2 = 산

array(['1', '2'], dtype=object)

In [27]:
age['spc_grd_info'].unique()

array(['일반', '산'], dtype=object)

In [28]:
age['co_bd_cd'].unique() #1 = 일반건축물, 2 = 집합건축물

array(['2', '1'], dtype=object)

In [29]:
age['co_bd_nm'].unique()

array(['집합건축물', '일반건축물'], dtype=object)

In [30]:
age['dong'].nunique()

1241

In [31]:
print(age['mus_cd'].unique(), age['mus_cd_nm'].unique(), sep='\n')

['14000' '02000' '03000' '01000' '04000' '11000' '20000' '10000' '16000'
 '07000' '19000' '09000' '17000' '21000' '18000' '06000' '13000' '05000'
 '15000' 'Z8000' '24000' '27000' 'Z5000' None '22000' '08000' '30000']
['업무시설' '공동주택' '제1종근린생활시설' '단독주택' '제2종근린생활시설' '노유자시설' '자동차관련시설' '교육연구시설'
 '위락시설' '판매시설' '위험물저장및처리시설' '의료시설' '공장' '동.식물 관련시설' '창고시설' '종교시설' '운동시설'
 '문화및집회시설' '숙박시설' '교육연구및복지시설' '방송통신시설' '관광휴게시설' None '분뇨.쓰레기처리시설' '운수시설']


In [32]:
num_cols = ['bd_h','grd_flr_num','ugr_flr_num']

age[num_cols].describe()

,bd_h,grd_flr_num,ugr_flr_num
count,20534.000,20534.000,20534.000
mean,4.868,2.663,0.209
std,10.590,3.221,0.436
min,0.000,0.000,0.000
25%,0.000,1.000,0.000
50%,0.000,2.000,0.000
75%,7.500,3.000,0.000
max,141.790,47.000,6.000


In [33]:
# 새로운 컬럼 (emd_jibun) 만들기

age['emd_jibun'] = age['emd_cd'].map(str) + '(' + age['jibun'] + ')'

age.head(1)

,emd_cd,emd_nm,spc_grd_cd,spc_grd_info,jibun,co_bd_cd,co_bd_nm,bd_nm,dong,bd_area,mus_cd,mus_cd_nm,bd_h,grd_flr_num,ugr_flr_num,athr_date,us_athr_date,bd_age,geometry,emd_jibun
0,2629010600,부산광역시 남구 대연동,1,일반,6-5,2,집합건축물,레이듐오피스텔,None,1830.380,14000,업무시설,29.400,9,1,2001-04-10,2002-07-26,20.000,"MULTIPOLYGON (((129.10670 35.13866, 129.10663 ...",2629010600(6-5)


In [34]:
# 불필요한 컬럼 제거

age_temp = age.drop(columns = ['emd_cd', 'emd_nm','jibun','athr_date'], axis=1) # 건물연령의 기준은 사용승인일자이므로 athr_date도 제거  

# us_athr_date 데이터타입 변경

age_temp['us_athr_date'] = pd.to_datetime(age_temp['us_athr_date'], infer_datetime_format=True, errors='coerce')

print(age_temp.dtypes)

# 컬럼 재배치

col_1 = age_temp.columns[-1:].to_list()
col_2 = age_temp.columns[:-1].to_list()
new_col = col_1 + col_2
cp_age = age_temp[new_col]
cp_age.head(1) # competition age

spc_grd_cd              object
spc_grd_info            object
co_bd_cd                object
co_bd_nm                object
bd_nm                   object
dong                    object
bd_area                float64
mus_cd                  object
mus_cd_nm               object
bd_h                   float64
grd_flr_num              int64
ugr_flr_num              int64
us_athr_date    datetime64[ns]
bd_age                 float64
geometry              geometry
emd_jibun               object
dtype: object


,emd_jibun,spc_grd_cd,spc_grd_info,co_bd_cd,co_bd_nm,bd_nm,dong,bd_area,mus_cd,mus_cd_nm,bd_h,grd_flr_num,ugr_flr_num,us_athr_date,bd_age,geometry
0,2629010600(6-5),1,일반,2,집합건축물,레이듐오피스텔,None,1830.380,14000,업무시설,29.400,9,1,2002-07-26,20.000,"MULTIPOLYGON (((129.10670 35.13866, 129.10663 ..."


In [35]:
cp_age.to_file(input_path.joinpath('대회_건축물연령_정리.geojson'), driver='GeoJSON')

---

## 도로명주소(건물)

* 현재 기준의 부산시 남구 건물의 도로명 주소 데이터

In [36]:
addr_bd.head(2)

,BDTYP_CD,BULD_NM,BULD_NM_DC,BULD_SE_CD,BUL_MAN_NO,EMD_CD,GRO_FLO_CO,LNBR_MNNM,LNBR_SLNO,UND_FLO_CO,geometry
0,11102,엔터모텔,None,0,28791,109,0,807,5,0,"MULTIPOLYGON (((129.06709 35.13758, 129.06721 ..."
1,15001,동명주유소,None,0,28790,109,0,807,4,0,"MULTIPOLYGON (((129.06721 35.13761, 129.06728 ..."


In [37]:
addr_bd.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 28858 entries, 0 to 28857
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   BDTYP_CD    28858 non-null  object  
 1   BULD_NM     6640 non-null   object  
 2   BULD_NM_DC  1885 non-null   object  
 3   BULD_SE_CD  28858 non-null  object  
 4   BUL_MAN_NO  28858 non-null  int64   
 5   EMD_CD      28858 non-null  object  
 6   GRO_FLO_CO  28858 non-null  int64   
 7   LNBR_MNNM   28858 non-null  int64   
 8   LNBR_SLNO   28858 non-null  int64   
 9   UND_FLO_CO  28858 non-null  int64   
 10  geometry    28858 non-null  geometry
dtypes: geometry(1), int64(5), object(5)
memory usage: 2.4+ MB


#### 컬럼별 설명

* BDTYP_CD:    건물용도코드
* BULD_NM:    건축물대장 건물명
* BULD_NM_DC:    상세건물명
* BULD_SE_CD:    건물 구분 코드
* BUL_MAN_NO:    건물 일련번호
* EMD_CD:    읍면동 코드
* GRO_FLO_CO:    건물의 지상층수
* LNBR_MNNM:    건물의 지번본번
* LNBR_SLNO:    건물의 지번부번
* UND_FLO_CO:    건물의 지하층수

In [38]:
addr_bd['EMD_CD'].unique() # 00을 붙여야 다른 dataframe과 연결하기 용이해보임

array(['109', '106', '111', '110', '108', '107'], dtype=object)

In [39]:
addr_bd['BULD_NM_DC'].nunique()

729

In [40]:
addr_bd['BUL_MAN_NO'].nunique()

28858

In [41]:
num_cols = ['GRO_FLO_CO','UND_FLO_CO']

addr_bd[num_cols].describe()

,GRO_FLO_CO,UND_FLO_CO
count,28858.000,28858.000
mean,0.783,0.051
std,2.755,0.268
min,0.000,0.000
25%,0.000,0.000
50%,0.000,0.000
75%,0.000,0.000
max,69.000,7.000


In [42]:
# emd_jb 컬럼 만들기 --- 2629010600(6-5)

addr_bd['emd_jibun'] = '26190' + addr_bd['EMD_CD'].map(str) + '00'  + '(' + addr_bd['LNBR_MNNM'].map(str) + '-' + addr_bd['LNBR_SLNO'].map(str) + ')'

addr_bd.head(1)

,BDTYP_CD,BULD_NM,BULD_NM_DC,BULD_SE_CD,BUL_MAN_NO,EMD_CD,GRO_FLO_CO,LNBR_MNNM,LNBR_SLNO,UND_FLO_CO,geometry,emd_jibun
0,11102,엔터모텔,None,0,28791,109,0,807,5,0,"MULTIPOLYGON (((129.06709 35.13758, 129.06721 ...",2619010900(807-5)


In [43]:
# 불필요한 컬럼 정리

addr_bd_temp = addr_bd.drop(columns = ['LNBR_MNNM','LNBR_SLNO','EMD_CD'], axis=1)

# 컬럼 재배치

col_1 = addr_bd_temp.columns[-1:].to_list()
col_2 = addr_bd_temp.columns[:-1].to_list()
new_col = col_1 + col_2
cp_addr_bd = addr_bd_temp[new_col]
cp_addr_bd.head(1) # competition addr_bd

,emd_jibun,BDTYP_CD,BULD_NM,BULD_NM_DC,BULD_SE_CD,BUL_MAN_NO,GRO_FLO_CO,UND_FLO_CO,geometry
0,2619010900(807-5),11102,엔터모텔,None,0,28791,0,0,"MULTIPOLYGON (((129.06709 35.13758, 129.06721 ..."


In [44]:
cp_addr_bd.to_file(input_path.joinpath('대회_도로명주소_건물_정리.geojson'), driver='GeoJSON')

---

## 공시지가

* 현재 기준의 부산시 남구 공시지가 데이터

In [45]:
prc.head(2)

,index,emd_cd,emd_nm,reg_cd,reg_nm,jibun,jimok,year,month,lnd_prc,stndrd_yn,jimok_cd,jimok_nm,lnd_area,date,geometry
0,2629010700105620003,2629010700,부산광역시 남구 용호동,1,일반,562-3,562-3대,2020,01,1017000,0,None,None,0.000,2021-05-18,"MULTIPOLYGON (((129.11309 35.10997, 129.11292 ..."
1,2629010700105620004,2629010700,부산광역시 남구 용호동,1,일반,562-4,562-4대,2020,01,960000,0,None,None,0.000,2021-05-18,"MULTIPOLYGON (((129.11326 35.11000, 129.11309 ..."


In [46]:
prc.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 45777 entries, 0 to 45776
Data columns (total 16 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   index      45777 non-null  object  
 1   emd_cd     45777 non-null  object  
 2   emd_nm     45777 non-null  object  
 3   reg_cd     45777 non-null  object  
 4   reg_nm     45777 non-null  object  
 5   jibun      45777 non-null  object  
 6   jimok      45777 non-null  object  
 7   year       45447 non-null  object  
 8   month      45447 non-null  object  
 9   lnd_prc    45777 non-null  int64   
 10  stndrd_yn  45447 non-null  object  
 11  jimok_cd   1131 non-null   object  
 12  jimok_nm   1131 non-null   object  
 13  lnd_area   45777 non-null  float64 
 14  date       45777 non-null  object  
 15  geometry   45777 non-null  geometry
dtypes: float64(1), geometry(1), int64(1), object(13)
memory usage: 5.6+ MB


#### 컬럼별 설명

* index: 해당 데이터의 고유번호
* emd_cd: 법정동코드
* emd_nm: 법정동의 이름
* reg_cd: 대장구분코드
* reg_nm: 대장구분 정보
* jibun: 지번 정보
* jimok: 지번지목 정보
* year: 해당 데이터의 기준년도
* month: 해당 데이터의 기준월
* lnd_prc: 해당 데이터의 개별공시지가
* stndrd_yn: 해당 데이터의 표준지여부
* jimok_cd: 지목의 코드
* jimok_nm: 지목 정보
* lnd_area: 토지의 면적
* date: 해당 데이터의 기준일

In [47]:
prc.isnull().sum()

index            0
emd_cd           0
emd_nm           0
reg_cd           0
reg_nm           0
jibun            0
jimok            0
year           330
month          330
lnd_prc          0
stndrd_yn      330
jimok_cd     44646
jimok_nm     44646
lnd_area         0
date             0
geometry         0
dtype: int64

In [48]:
prc['stndrd_yn'].unique() # 0: NO, 1: YES

array(['0', None, '1'], dtype=object)

In [49]:
prc['stndrd_yn'].isnull().sum()

330

In [50]:
print(prc['jimok_cd'].unique(), prc['jimok_nm'].unique(), sep='\n')

[None '05' '08' '01' '10' '09' '11' '28' '12' '13' '02' '25']
[None '임야' '대' '전' '학교용지' '공장용지' '주차장' '잡종지' '주유소용지' '창고용지' '답' '종교용지']


In [51]:
print(prc['year'].unique(), prc['month'].unique())

['2020' None] ['01' None '07']


In [52]:
# emd_jb 컬럼 만들기 --- 2629010600(6-5)

prc['emd_jibun'] = prc['emd_cd'].map(str) +  '(' + prc['jibun'] + ')'
prc.head(1)

,index,emd_cd,emd_nm,reg_cd,reg_nm,jibun,jimok,year,month,lnd_prc,stndrd_yn,jimok_cd,jimok_nm,lnd_area,date,geometry,emd_jibun
0,2629010700105620003,2629010700,부산광역시 남구 용호동,1,일반,562-3,562-3대,2020,01,1017000,0,None,None,0.000,2021-05-18,"MULTIPOLYGON (((129.11309 35.10997, 129.11292 ...",2629010700(562-3)


In [53]:
# 새로운 컬럼 추가 (기준연월: std_date)

prc['year'] = prc['year'].replace('None', np.nan) # 'None'값 null로 대체
prc['month'] = prc['month'].replace('None', np.nan)

In [54]:
prc['std_date'] = prc['year'].map(str) + '-' + prc['month'].map(str)
prc['std_date'] = prc['std_date'].replace('None-None', np.nan)

print(prc['std_date'].unique(), prc['std_date'].isnull().sum(), sep='\n')

['2020-01' nan '2020-07']
330


In [55]:
prc.head(1)

,index,emd_cd,emd_nm,reg_cd,reg_nm,jibun,jimok,year,month,lnd_prc,stndrd_yn,jimok_cd,jimok_nm,lnd_area,date,geometry,emd_jibun,std_date
0,2629010700105620003,2629010700,부산광역시 남구 용호동,1,일반,562-3,562-3대,2020,01,1017000,0,None,None,0.000,2021-05-18,"MULTIPOLYGON (((129.11309 35.10997, 129.11292 ...",2629010700(562-3),2020-01


In [56]:
# 불필요한 컬럼 정리

prc_temp = prc.drop(columns = ['index','emd_cd','emd_nm','jibun','year','month'], axis=1)

In [57]:
# 컬럼 재배치

col_1 = prc_temp.columns[-2:].to_list()
col_2 = prc_temp.columns[:-2].to_list()
new_col = col_1 + col_2
cp_prc = prc_temp[new_col]
cp_prc.head(1) # competition addr_bd

,emd_jibun,std_date,reg_cd,reg_nm,jimok,lnd_prc,stndrd_yn,jimok_cd,jimok_nm,lnd_area,date,geometry
0,2629010700(562-3),2020-01,1,일반,562-3대,1017000,0,None,None,0.000,2021-05-18,"MULTIPOLYGON (((129.11309 35.10997, 129.11292 ..."


In [58]:
cp_prc.to_file(input_path.joinpath('대회_공시지가_정리.geojson'), driver='GeoJSON')

---

## 공공데이터 활용하여 결측값 채우기/데이터 업데이트

* 데이터를 결합하는 데에 소요되는 시간을 줄이기 위해 컬럼 수 정리 및 데이터 전처리를 우선적으로 진행

### 공공데이터 불러오기 (건축물연령)

In [59]:
public_age = gpd.read_file(input_path.joinpath('건축물연령_(210114기준).shp'), encoding='CP949')
public_age.head(1)

,A0,A1,A2,A3,A4,A5,A6,A7,A8,A9,...,A22,A23,A24,A25,A26,A27,A28,A29,A30,geometry
0,32457038,2002391922631842049200000000,2629010600100060005,2629010600,부산광역시 남구 대연동,1,일반,6-5,13176,2,...,1,2001-04-10,2002-07-26,20.000,03,20대,025,25세미만,2021-01-13,"POLYGON ((391934.193 184199.828, 391927.662 18..."


In [60]:
public_age.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 20534 entries, 0 to 20533
Data columns (total 32 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   A0        20534 non-null  int64   
 1   A1        20534 non-null  object  
 2   A2        20534 non-null  object  
 3   A3        20534 non-null  object  
 4   A4        20534 non-null  object  
 5   A5        20534 non-null  object  
 6   A6        20534 non-null  object  
 7   A7        20534 non-null  object  
 8   A8        20534 non-null  object  
 9   A9        20534 non-null  object  
 10  A10       20534 non-null  object  
 11  A11       20534 non-null  object  
 12  A12       20534 non-null  object  
 13  A13       7177 non-null   object  
 14  A14       2429 non-null   object  
 15  A15       20534 non-null  float64 
 16  A16       20529 non-null  object  
 17  A17       20527 non-null  object  
 18  A18       20529 non-null  object  
 19  A19       20524 non-null  object  
 20

#### 컬럼별 내용

* A0 도형ID 개방DB에서 정의한 도형ID 정보

* A1 GIS건물통합식별번호 건물공간정보DB 식별자 (층별번호, 호별번호는 '0'으로 통일함)

* A2 고유번호 각 필지를 서로 구별하기 위하여 필지마다 붙이는 고유한 번호 

* A3 법정동코드 토지가 소재한 행정구역코드(법정동코드) 10자리

* A4 법정동명 토지가 소재한 소재지의 행정구역 명칭(법정동명)

* A5 특수지구분코드 해당 필지의 특수지구분 코드

* A6 특수지구분명 해당 필지의 특수지구분 명칭

* A7 지번 필지에 부여하여 지적공부에 등록한 번호. 지번본번과 지번부번으로 구성

* A8 건물식별번호 건축물대장을 식별할 수 있는 고유번호

* A9 집합건물구분코드 건축물대장에서 일반건축물/집합건축물인지 구분하는 코드

* A10 집합건물구분 건축물대장에서 일반건축물/집합건축물인지 구분하는 코드정보

* A11대장종류코드 건축물대장의 종류를 구분하는 코드

* A12 대장종류 건축물대장의 종류를 구분하는 코드정보

* A13 건물명 건물의 명칭 

* A14 건물동명 건축물의 동 명칭 정보

* A15 건물연면적 건축물에서 지하층을 포함한 건물 각 층의 바닥면적 합산한 면적(㎡)

* A16 건축물구조코드 건축물 구조에 대한 분류코드

* A17 건축물구조명 건축물 구조에 대한 분류 코드정보

* A18 주요용도코드 건축물의 이용 목적 및 형태를 알 수 있는 주요 용도에 대한 분류 코드

* A19 주요용도명 건축물의 이용 목적 및 형태를 알 수 있는 주요 용도에 대한 분류 코드정보

* A20 건물높이 건축물에 대한 높이(m)

* A21 지상층수 건축물에 대한 지상으로 되어 있는 층수

* A22 지하층수 건축물에 대한 지하으로 되어 있는 층수

* A23 허가일자 건축물의 건축 허가일자

* A24 사용승인일자 건축물의 사용 승인 일자

* A25 건물연령 건축물의 연령

* A26 연령대구분코드 10단위 세대 구분코드

* A27 연령대구분명 10단위 세대 구분 코드정보

* A28 연령대5계급코드 5단위 세대 구분코드

* A29 연령대5계급명 5단위 세대 구분 코드정보

* A30 데이터기준일자 데이터 작성 기준일자


In [61]:
public_age['A6'].unique() # 1: 일반, 2: 산

array(['일반', '산'], dtype=object)

In [62]:
public_age['A10'].unique() # 1: 일반, 2: 집합

array(['집합건축물', '일반건축물'], dtype=object)

In [63]:
public_age['A11'].unique() # 2: 일반건축물대장, 3: 표제부

array(['3', '2'], dtype=object)

In [64]:
public_age['A12'].unique() # 2: 일반건축물대장, 3: 표제부

array(['표제부', '일반건축물대방'], dtype=object)

In [65]:
print(public_age['A16'].unique(), public_age['A17'].unique(), sep='\n')

['21' '11' '51' '12' '31' '32' '42' '19' '52' '99' '33' '13' '41' '63'
 None '39' '29' '10' '49' '61' '22']
['철근콘크리트구조' '벽돌구조' '일반목구조' '블록구조' '일반철골구조' '경량철골구조' '철골철근콘크리트구조' '기타조적구조'
 '통나무구조' '기타구조' '강파이프구조' '석구조' '철골콘크리트구조' None '기타강구조' '기타콘크리트구조' '조적구조'
 '기타철골철근콘크리트구조' '프리케스트콘크리트구조']


In [66]:
# 불필요한 컬럼 제거
public_age_dropped = public_age.drop(columns=['A0', 'A1', 'A2','A23','A26','A27','A28','A29','A30' ])

In [67]:
public_age_dropped.columns

Index(['A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'A11', 'A12', 'A13',
       'A14', 'A15', 'A16', 'A17', 'A18', 'A19', 'A20', 'A21', 'A22', 'A24',
       'A25', 'geometry'],
      dtype='object')

In [68]:
# 컬럼명 변경
public_age_dropped.columns = ['emd_cd','emd_nm','spc_grd_cd','spc_grd_info','jibun','buld_se_cd','co_bd_cd','co_bd_nm',
                              'reg_cd','reg_nm','bd_nm','dong','bd_area','str_cd','str_nm','mus_cd',
                              'mus_cd_nm','bd_h','grd_flr_num','ugr_flr_num','us_athr_date','bd_age','geometry'
                             ]

In [69]:
public_age_dropped.head(1)

,emd_cd,emd_nm,spc_grd_cd,spc_grd_info,jibun,buld_se_cd,co_bd_cd,co_bd_nm,reg_cd,reg_nm,...,str_cd,str_nm,mus_cd,mus_cd_nm,bd_h,grd_flr_num,ugr_flr_num,us_athr_date,bd_age,geometry
0,2629010600,부산광역시 남구 대연동,1,일반,6-5,13176,2,집합건축물,3,표제부,...,21,철근콘크리트구조,14000,업무시설,29.400,9,1,2002-07-26,20.000,"POLYGON ((391934.193 184199.828, 391927.662 18..."


In [70]:
public_age_dropped.dtypes

emd_cd            object
emd_nm            object
spc_grd_cd        object
spc_grd_info      object
jibun             object
buld_se_cd        object
co_bd_cd          object
co_bd_nm          object
reg_cd            object
reg_nm            object
bd_nm             object
dong              object
bd_area          float64
str_cd            object
str_nm            object
mus_cd            object
mus_cd_nm         object
bd_h             float64
grd_flr_num        int64
ugr_flr_num        int64
us_athr_date      object
bd_age           float64
geometry        geometry
dtype: object

In [71]:
# us_athr_date 데이터 타입 변경

public_age_dropped['us_athr_date'] = pd.to_datetime(public_age_dropped['us_athr_date'], infer_datetime_format=True, errors='coerce')
print(public_age_dropped.dtypes)
public_age_dropped.head(1)

emd_cd                  object
emd_nm                  object
spc_grd_cd              object
spc_grd_info            object
jibun                   object
buld_se_cd              object
co_bd_cd                object
co_bd_nm                object
reg_cd                  object
reg_nm                  object
bd_nm                   object
dong                    object
bd_area                float64
str_cd                  object
str_nm                  object
mus_cd                  object
mus_cd_nm               object
bd_h                   float64
grd_flr_num              int64
ugr_flr_num              int64
us_athr_date    datetime64[ns]
bd_age                 float64
geometry              geometry
dtype: object


,emd_cd,emd_nm,spc_grd_cd,spc_grd_info,jibun,buld_se_cd,co_bd_cd,co_bd_nm,reg_cd,reg_nm,...,str_cd,str_nm,mus_cd,mus_cd_nm,bd_h,grd_flr_num,ugr_flr_num,us_athr_date,bd_age,geometry
0,2629010600,부산광역시 남구 대연동,1,일반,6-5,13176,2,집합건축물,3,표제부,...,21,철근콘크리트구조,14000,업무시설,29.400,9,1,2002-07-26,20.000,"POLYGON ((391934.193 184199.828, 391927.662 18..."


In [72]:
# 공통된 코드 구성 (emd_jb)

#public_age_dropped['emd_cd'][0] + '(' + public_age_dropped['jibun'][0] + ')'

public_age_dropped['emd_jibun'] = public_age_dropped['emd_cd'].map(str) + "(" + public_age_dropped['jibun'] + ")"

public_age_dropped.head(1)

,emd_cd,emd_nm,spc_grd_cd,spc_grd_info,jibun,buld_se_cd,co_bd_cd,co_bd_nm,reg_cd,reg_nm,...,str_nm,mus_cd,mus_cd_nm,bd_h,grd_flr_num,ugr_flr_num,us_athr_date,bd_age,geometry,emd_jibun
0,2629010600,부산광역시 남구 대연동,1,일반,6-5,13176,2,집합건축물,3,표제부,...,철근콘크리트구조,14000,업무시설,29.400,9,1,2002-07-26,20.000,"POLYGON ((391934.193 184199.828, 391927.662 18...",2629010600(6-5)


In [73]:
# 컬럼 추가적으로 drop

public_age_dropped_2 = public_age_dropped.drop(columns=['emd_cd','emd_nm','jibun'])

In [74]:
# 컬럼 재배치

col_1 = public_age_dropped_2.columns[-1:].to_list()
col_2 = public_age_dropped_2.columns[:-1].to_list()
new_col = col_1 + col_2
pb_age = public_age_dropped_2[new_col]
pb_age.head()

,emd_jibun,spc_grd_cd,spc_grd_info,buld_se_cd,co_bd_cd,co_bd_nm,reg_cd,reg_nm,bd_nm,dong,...,str_cd,str_nm,mus_cd,mus_cd_nm,bd_h,grd_flr_num,ugr_flr_num,us_athr_date,bd_age,geometry
0,2629010600(6-5),1,일반,13176,2,집합건축물,3,표제부,레이듐오피스텔,None,...,21,철근콘크리트구조,14000,업무시설,29.400,9,1,2002-07-26,20.000,"POLYGON ((391934.193 184199.828, 391927.662 18..."
1,2629010600(6-8),1,일반,100190333,2,집합건축물,3,표제부,대박세븐,None,...,21,철근콘크리트구조,02000,공동주택,17.400,6,0,2012-07-26,10.000,"POLYGON ((391886.066 184178.783, 391885.197 18..."
2,2629010600(6-10),1,일반,3491,1,일반건축물,2,일반건축물대방,None,None,...,21,철근콘크리트구조,03000,제1종근린생활시설,0.000,5,1,1988-11-17,34.000,"POLYGON ((391910.721 184198.435, 391904.682 18..."
3,2629010600(6-27),1,일반,13177,2,집합건축물,3,표제부,유니온 해오름,유니온 해오름,...,21,철근콘크리트구조,02000,공동주택,13.200,5,0,2004-02-26,18.000,"POLYGON ((391878.562 184154.045, 391866.311 18..."
4,2629010600(6-28),1,일반,100192746,2,집합건축물,3,표제부,채송화뜰,None,...,21,철근콘크리트구조,02000,공동주택,15.800,6,0,2013-02-28,9.000,"POLYGON ((391887.775 184164.916, 391881.596 18..."


In [75]:
#  공공데이터_건물연령정보 저장
pb_age.to_file(input_path.joinpath('공공데이터_건물연령정보.geojson'), driver='GeoJSON')

---

### 공공데이터 불러오기 (개별공시지가)

In [76]:
public_price = gpd.read_file(input_path.joinpath('개별공시지가정보_(210813기준).shp'), encoding='CP949') # 부산광역시 기준 --- 남구만 추출할 필요 있음
public_price.head()

,A0,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,geometry
0,2614010300102540025,2614010300,부산광역시 서구 동대신동3가,1,일반,254-25,254-25대,2021,01,632400,0,None,None,0.000,2021-08-10,"POLYGON ((384088.172 181547.528, 384094.391 18..."
1,2614010300102580191,2614010300,부산광역시 서구 동대신동3가,1,일반,258-191,258-191대,2021,01,831600,0,None,None,0.000,2021-08-10,"POLYGON ((384245.172 181550.040, 384247.105 18..."
2,2614010300102580062,2614010300,부산광역시 서구 동대신동3가,1,일반,258-62,258-62대,2021,01,300600,0,None,None,0.000,2021-08-10,"POLYGON ((384244.841 181548.518, 384249.906 18..."
3,2614010300102540109,2614010300,부산광역시 서구 동대신동3가,1,일반,254-109,254-109도,2021,01,204600,0,None,None,0.000,2021-08-10,"POLYGON ((384115.243 181547.764, 384119.151 18..."
4,2614010300102580020,2614010300,부산광역시 서구 동대신동3가,1,일반,258-20,258-20대,2021,01,831600,0,None,None,0.000,2021-08-10,"POLYGON ((384252.049 181550.337, 384256.710 18..."


#### 컬럼별 내용


* A0 고유번호 각 필지를 서로 구별하기 위하여 필지마다 붙이는 고유한 번호

* A1 법정동코드 토지가 소재한 행정구역코드(법정동코드) 10자리

* A2 법정동명 토지가 소재한 소재지의 행정구역 명칭(법정동명)

* A3 대장구분코드 토지가 위치한 토지의 대장 구분 (토지(임야)대장구분)코드

* A4 대장구분명 토지가 위치한 토지의 대장 구분 (토지(임야)대장구분)

* A5 지번 필지에 부여하여 지적공부에 등록한 번호

* A6 지번지목부호 지번지목부호

* A7 기준연도 개별공시지가의 기준연도

* A8 기준월 개별공시지가의 기준월

* A9 개별공시지가 대한민국의 건설교통부가 토지의 가격을 조사, 감정을 해 공시함. 개별토지에한 공시 가격(원/㎡)

* A10 표준지여부 대상 토지를 평가할 때, 평가의 기준으로 삼는 필지의 여부

* A11 지목코드 지목코드

* A12 지목 지목

* A13 토지면적 토지면적(㎡)

* A14 데이터 작성 기준일자

In [77]:
print(public_price['A3'].unique(), public_price['A4'].unique(), public_price['A3'].value_counts(), sep='\n') # 1:일반, 2: 산,  4: 가지번(부번세분), 6: 블럭지번(롯트세분),

['1' '2' '6' '4']
['일반' '산' '블럭지번(롯트세분)' '가지번(부번세분)']
1    687852
2     33337
6       221
4        12
Name: A3, dtype: int64


##### Infos

* 블럭지번(롯트세분): 아직 지번/지목이 지정되지 않은 상태에서 임시적으로 주어지는 부여되는 번호 === 분석에서 제외할 수 있음 (현재 이미 개발 중이므로)

* 가지번(부번세분): '가지번'의 개념에 대한 명확한 이해는 실패

  - 하지만 해당 토지의 지목이 '답' (물을 상시적으로 사용하는 땅)인 것으로 보아 분석에서 제외하는 것이 나을 것으로 예상 됨

In [78]:
print(public_price['A11'].unique(), public_price['A12'].unique(), sep='\n')

[None '08' '05' '01' '10' '09' '02' '04' '28' '22' '11' '15' '13' '12'
 '25' '24' '21' '06' '17' '03' '23' '27']
[None '대' '임야' '전' '학교용지' '공장용지' '답' '목장용지' '잡종지' '공원' '주차장' '철도용지' '창고용지'
 '주유소용지' '종교용지' '유원지' '수도용지' '광천지' '하천' '과수원' '체육용지' '묘지']


In [79]:
public_price['A7'].unique()

array(['2021', None], dtype=object)

In [80]:
public_price['A8'].unique()

array(['01', None], dtype=object)

In [81]:
# 새로운 컬럼 추가 (기준연월: std_date)

public_price['A7'] = public_price['A7'].replace('None', np.nan)
public_price['A8'] = public_price['A8'].replace('None', np.nan)

public_price['std_date'] = public_price['A7'].map(str) + '-' + public_price['A8'].map(str)
public_price['std_date'] = public_price['std_date'].replace('None-None', np.nan)

print(public_price['std_date'].unique(), public_price['std_date'].isnull().sum(), sep='\n')

['2021-01' nan]
20612


In [82]:
# 불필요한 컬럼 제거

public_price_dropped = public_price.drop(columns=['A0','A7','A8'])

# 부산광역시 남구 데이터만 남기기

is_namgu = public_price_dropped['A2'].str.contains('남구')
public_price_namgu = public_price_dropped[is_namgu]

In [83]:
public_price_namgu.head(1)

,A1,A2,A3,A4,A5,A6,A9,A10,A11,A12,A13,A14,geometry,std_date
28870,2629010900,부산광역시 남구 문현동,1,일반,332-46,332-46도,11200,0,None,None,0.000,2021-08-10,"POLYGON ((389142.963 183158.099, 389137.662 18...",2021-01


In [84]:
# 새로운 컬럼 생성

public_price_namgu['emd_jibun'] = public_price_namgu['A1'].map(str) + "(" + public_price_namgu['A5'] + ")"

public_price_namgu.head(1)

,A1,A2,A3,A4,A5,A6,A9,A10,A11,A12,A13,A14,geometry,std_date,emd_jibun
28870,2629010900,부산광역시 남구 문현동,1,일반,332-46,332-46도,11200,0,None,None,0.000,2021-08-10,"POLYGON ((389142.963 183158.099, 389137.662 18...",2021-01,2629010900(332-46)


In [85]:
# 컬럼 추가적으로 drop

public_price_namgu_2 = public_price_namgu.drop(columns=['A1','A2','A5'])

In [86]:
public_price_namgu_2.head(1)

,A3,A4,A6,A9,A10,A11,A12,A13,A14,geometry,std_date,emd_jibun
28870,1,일반,332-46도,11200,0,None,None,0.000,2021-08-10,"POLYGON ((389142.963 183158.099, 389137.662 18...",2021-01,2629010900(332-46)


In [87]:
# 컬럼명 변경

public_price_namgu_2.columns = ['reg_cd','reg_nm','jimok','lnd_prc','stndrd_yn','jimok_cd','jimok_nm','lnd_area','date','geometry','std_date','emd_jibun']
print(public_price_namgu_2.columns)

Index(['reg_cd', 'reg_nm', 'jimok', 'lnd_prc', 'stndrd_yn', 'jimok_cd',
       'jimok_nm', 'lnd_area', 'date', 'geometry', 'std_date', 'emd_jibun'],
      dtype='object')


In [88]:
# 컬럼 재배치

col_1 = public_price_namgu_2.columns[-2:][::-1].to_list()
col_2 = public_price_namgu_2.columns[:-2].to_list()
new_col = col_1 + col_2
pb_prc = public_price_namgu_2[new_col]
pb_prc.head(1)

,emd_jibun,std_date,reg_cd,reg_nm,jimok,lnd_prc,stndrd_yn,jimok_cd,jimok_nm,lnd_area,date,geometry
28870,2629010900(332-46),2021-01,1,일반,332-46도,11200,0,None,None,0.000,2021-08-10,"POLYGON ((389142.963 183158.099, 389137.662 18..."


In [89]:
pb_prc.isnull().sum()

emd_jibun        0
std_date       239
reg_cd           0
reg_nm           0
jimok            0
lnd_prc          0
stndrd_yn      239
jimok_cd     44561
jimok_nm     44561
lnd_area         0
date             0
geometry         0
dtype: int64

In [90]:
pb_prc.to_file(input_path.joinpath('공공데이터_개별공시지가.geojson'), driver='GeoJSON')

---

### 공공데이터 불러오기 (표준공시지가정보)

* 각종 지형관련 정보를 활용
    
    - 그러므로 가격 정보 및 연도 등은 제외하도록 한다

In [91]:
stdprice = pd.read_csv(input_path.joinpath('부산광역시_표준공시지가정보.csv'), encoding='CP949')
print(stdprice.columns)
stdprice.head(2)

Index(['고유번호', '법정동코드', '법정동명', '특수지구분코드', '특수지구분명', '지번', '표준지일련번호', '기준연도',
       '사업지구면적', '지목코드', '지목', '실제지목코드', '실제지목', '토지면적', '용도지역코드1', '용도지역명1',
       '용도지역코드2', '용도지역명2', '용도지구코드1', '용도지구명1', '용도지구코드2', '용도지구명2', '저촉율',
       '토지이용상황코드', '토지이용상황', '지형높이코드', '지형높이', '지형형상코드', '지형형상', '도로측면코드',
       '도로측면', '도로거리코드', '도로거리', '공시지가', '표준지소유구분코드', '표준지소유구분명', '소유형태코드',
       '소유형태명', '데이터기준일자'],
      dtype='object')


,고유번호,법정동코드,법정동명,특수지구분코드,특수지구분명,지번,표준지일련번호,기준연도,사업지구면적,지목코드,...,도로측면코드,도로측면,도로거리코드,도로거리,공시지가,표준지소유구분코드,표준지소유구분명,소유형태코드,소유형태명,데이터기준일자
0,2611010100100020130,2611010100,부산광역시 중구 영주동,1,일반,2-130,1,2012,0,8,...,10,세로한면(불),0,지정되지않음,555000,nan,nan,0,지정되지않음,2016-11-08
1,2611010100100020130,2611010100,부산광역시 중구 영주동,1,일반,2-130,1,2013,0,8,...,10,세로한면(불),0,지정되지않음,570000,nan,nan,0,지정되지않음,2016-11-08


In [92]:
stdprice.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181672 entries, 0 to 181671
Data columns (total 39 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   고유번호       181672 non-null  object 
 1   법정동코드      181672 non-null  int64  
 2   법정동명       181672 non-null  object 
 3   특수지구분코드    181672 non-null  int64  
 4   특수지구분명     181672 non-null  object 
 5   지번         181308 non-null  object 
 6   표준지일련번호    181672 non-null  int64  
 7   기준연도       181672 non-null  int64  
 8   사업지구면적     181672 non-null  int64  
 9   지목코드       181672 non-null  int64  
 10  지목         181672 non-null  object 
 11  실제지목코드     181672 non-null  int64  
 12  실제지목       8167 non-null    object 
 13  토지면적       181672 non-null  float64
 14  용도지역코드1    181672 non-null  int64  
 15  용도지역명1     181672 non-null  object 
 16  용도지역코드2    181672 non-null  int64  
 17  용도지역명2     181672 non-null  object 
 18  용도지구코드1    181672 non-null  int64  
 19  용도지구명1     181672 non-n

In [93]:
# '남구'인 데이터만 남기기

is_namgu = stdprice['법정동명'].str.contains('남구')
stdprice_namgu = stdprice[is_namgu]

stdprice_namgu.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11400 entries, 75469 to 86868
Data columns (total 39 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   고유번호       11400 non-null  object 
 1   법정동코드      11400 non-null  int64  
 2   법정동명       11400 non-null  object 
 3   특수지구분코드    11400 non-null  int64  
 4   특수지구분명     11400 non-null  object 
 5   지번         11400 non-null  object 
 6   표준지일련번호    11400 non-null  int64  
 7   기준연도       11400 non-null  int64  
 8   사업지구면적     11400 non-null  int64  
 9   지목코드       11400 non-null  int64  
 10  지목         11400 non-null  object 
 11  실제지목코드     11400 non-null  int64  
 12  실제지목       128 non-null    object 
 13  토지면적       11400 non-null  float64
 14  용도지역코드1    11400 non-null  int64  
 15  용도지역명1     11400 non-null  object 
 16  용도지역코드2    11400 non-null  int64  
 17  용도지역명2     11400 non-null  object 
 18  용도지구코드1    11400 non-null  int64  
 19  용도지구명1     11400 non-null  object 
 20  용도

In [94]:
stdprice_namgu['표준지일련번호'].nunique()

1174

In [95]:
stdprice_namgu['기준연도'].unique() 

array([2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021])

In [96]:
stdprice_namgu['사업지구면적'].nunique()

1

In [97]:
print(stdprice_namgu['지목코드'].unique(), stdprice_namgu['지목'].unique(), sep='\n')

[ 8 11  5 10  2  1  9 28 12 13 25]
['대' '주차장' '임야' '학교용지' '답' '전' '공장용지' '잡종지' '주유소용지' '창고용지' '종교용지']


In [98]:
print(stdprice_namgu['실제지목코드'].unique(), stdprice_namgu['실제지목'].unique(), sep='\n')

[ 0  8 11 10  5]
[nan '대' '주차장' '학교용지' '임야']


In [99]:
print(stdprice_namgu['용도지역코드1'].unique(), stdprice_namgu['용도지역명1'].unique(), sep='\n')

[16 22 14 15 43 13 33 23 41 31]
['준주거지역' '일반상업지역' '제2종일반주거지역' '제3종일반주거지역' '자연녹지지역' '제1종일반주거지역' '준공업지역'
 '근린상업지역' '보전녹지지역' '전용공업지역']


In [100]:
print(stdprice_namgu['용도지역코드2'].unique(), stdprice_namgu['용도지역명2'].unique(), sep='\n')

[ 0 14]
['지정되지않음' '제2종일반주거지역']


In [101]:
print(stdprice_namgu['용도지구코드1'].unique(), stdprice_namgu['용도지구명1'].unique(), sep='\n')

[  0  28 300  18  29  99  25 160 130  14 110  17  24 150 120 521  20]
['지정되지않음' '방화지구' '학교시설보호지구' '학교시설보호지구(특정용도제한지구)' '최저고도지구' '기타지구'
 '역사문화미관지구' '역사문화미관지구(특화경관지구)' '특화경관지구(역사문화미관지구)' '특화경관지구' '자연경관지구'
 '자연경관지구(자연경관지구)' '기타경관지구' '중심지미관지구(시가지경관지구)' '중심지미관지구' '시가지경관지구(중심지미관지구)'
 '시가지경관지구' '중요시설물(항만)보호지구' '항만시설보호지구(중요시설보호지구)' '항만시설보호지구']


In [102]:
print(stdprice_namgu['용도지구코드2'].unique(), stdprice_namgu['용도지구명2'].unique(), sep='\n')

[  0  28  25 300  99]
['지정되지않음' '방화지구' '역사문화미관지구' '기타지구']


In [103]:
print(stdprice_namgu['토지이용상황코드'].unique(), stdprice_namgu['토지이용상황'].unique(), sep='\n')

[110 210 330 320 310 160 130 150 230 240 140 730 120 220 510 720 410 850
 430 420]
['단독' '상업용' '주상기타' '주상나지' '주상용' '주거기타' '다세대' '주거나지' '상업나지' '상업기타' '아파트'
 '토지임야' '연립' '업무용' '전' '자연림' '공업용' '공원묘지' '공업기타' '공업나지']


In [104]:
print(stdprice_namgu['지형높이코드'].unique(), stdprice_namgu['지형높이'].unique(), sep='\n')

[2 3 1 4]
['평지' '완경사' '저지' '급경사']


In [105]:
print(stdprice_namgu['지형형상코드'].unique(), stdprice_namgu['지형형상'].unique(), sep='\n')

[7 8 6 5 4 3 2 1]
['부정형' '자루형' '사다리형' '세로장방' '가로장방' '정방형' '삼각형']


In [106]:
print(stdprice_namgu['도로측면코드'].unique(), stdprice_namgu['도로측면'].unique(), sep='\n')

[10  8  2  9  1  7  6  3  4  5 12 11]
['세로한면(불)' '세로한면(가)' '광대소각' '세로각지(가)' '광대로한면' '소로각지' '소로한면' '광대세각' '중로한면'
 '중로각지' '맹지' '세로각지(불)']


In [107]:
print(stdprice_namgu['도로거리코드'].unique(), stdprice_namgu['도로거리'].unique(), sep='\n')

[0]
['지정되지않음']


In [108]:
print(stdprice_namgu['소유형태코드'].unique(), stdprice_namgu['소유형태명'].unique(), sep='\n')

[0 1 2]
['지정되지않음' '단독소유' '공동소유']


In [109]:
stdprice_namgu['저촉율'].mean()

6.408508771929824

In [110]:
# 중복되는 필지 제거 ('기준연도' 기준으로 최신 자료만 남기기)

stdprice_namgu_stnrd = stdprice_namgu.sort_values(by=['표준지일련번호', '기준연도'])
stdprice_cleansed = stdprice_namgu_stnrd.groupby('표준지일련번호').nth(-1).reset_index(drop=True)

stdprice_cleansed.head(2)

,고유번호,법정동코드,법정동명,특수지구분코드,특수지구분명,지번,기준연도,사업지구면적,지목코드,지목,...,도로측면코드,도로측면,도로거리코드,도로거리,공시지가,표준지소유구분코드,표준지소유구분명,소유형태코드,소유형태명,데이터기준일자
0,2629010600100080005,2629010600,부산광역시 남구 대연동,1,일반,8-5,2021,0,8,대,...,8,세로한면(가),0,지정되지않음,1625000,nan,nan,0,지정되지않음,2021-05-07
1,2629010600100160002,2629010600,부산광역시 남구 대연동,1,일반,16-2,2021,0,8,대,...,2,광대소각,0,지정되지않음,5420000,nan,nan,0,지정되지않음,2021-05-07


In [111]:
print(stdprice_cleansed.기준연도.unique(), stdprice_cleansed.기준연도.value_counts(), sep='\n')

[2021 2015]
2021    1132
2015      42
Name: 기준연도, dtype: int64


In [112]:
# 불필요한 컬럼 제거

stdprice_dropped = stdprice_cleansed.drop(columns=['고유번호','기준연도','사업지구면적','실제지목코드',
                                               '실제지목','용도지역코드2','용도지역명2','용도지구코드2','용도지구명2','도로거리코드','도로거리',
                                               '표준지소유구분코드','표준지소유구분명'])

stdprice_dropped.head(1)

,법정동코드,법정동명,특수지구분코드,특수지구분명,지번,지목코드,지목,토지면적,용도지역코드1,용도지역명1,...,지형높이코드,지형높이,지형형상코드,지형형상,도로측면코드,도로측면,공시지가,소유형태코드,소유형태명,데이터기준일자
0,2629010600,부산광역시 남구 대연동,1,일반,8-5,8,대,235.000,16,준주거지역,...,2,평지,6,자루형,8,세로한면(가),1625000,0,지정되지않음,2021-05-07


In [113]:
stdprice_dropped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1174 entries, 0 to 1173
Data columns (total 25 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   법정동코드     1174 non-null   int64  
 1   법정동명      1174 non-null   object 
 2   특수지구분코드   1174 non-null   int64  
 3   특수지구분명    1174 non-null   object 
 4   지번        1174 non-null   object 
 5   지목코드      1174 non-null   int64  
 6   지목        1174 non-null   object 
 7   토지면적      1174 non-null   float64
 8   용도지역코드1   1174 non-null   int64  
 9   용도지역명1    1174 non-null   object 
 10  용도지구코드1   1174 non-null   int64  
 11  용도지구명1    1174 non-null   object 
 12  저촉율       1174 non-null   int64  
 13  토지이용상황코드  1174 non-null   int64  
 14  토지이용상황    1174 non-null   object 
 15  지형높이코드    1174 non-null   int64  
 16  지형높이      1174 non-null   object 
 17  지형형상코드    1174 non-null   int64  
 18  지형형상      1174 non-null   object 
 19  도로측면코드    1174 non-null   int64  
 20  도로측면      1174 non-null   obje

In [114]:
stdprice_dropped['emd_jibun'] = stdprice_dropped['법정동코드'].map(str) + "(" + stdprice_dropped['지번'] + ")"

stdprice_dropped.head(1)

,법정동코드,법정동명,특수지구분코드,특수지구분명,지번,지목코드,지목,토지면적,용도지역코드1,용도지역명1,...,지형높이,지형형상코드,지형형상,도로측면코드,도로측면,공시지가,소유형태코드,소유형태명,데이터기준일자,emd_jibun
0,2629010600,부산광역시 남구 대연동,1,일반,8-5,8,대,235.000,16,준주거지역,...,평지,6,자루형,8,세로한면(가),1625000,0,지정되지않음,2021-05-07,2629010600(8-5)


In [115]:
# 컬럼 추가적으로 drop

stdprice_dropped_2 = stdprice_dropped.drop(columns=['법정동코드','법정동명','지번'])

# 컬럼 재배치

col_1 = stdprice_dropped_2.columns[-1:].to_list()
col_2 = stdprice_dropped_2.columns[:-1].to_list()
new_col = col_1 + col_2
pb_std_prc = stdprice_dropped_2[new_col]
pb_std_prc.head(1)

,emd_jibun,특수지구분코드,특수지구분명,지목코드,지목,토지면적,용도지역코드1,용도지역명1,용도지구코드1,용도지구명1,...,지형높이코드,지형높이,지형형상코드,지형형상,도로측면코드,도로측면,공시지가,소유형태코드,소유형태명,데이터기준일자
0,2629010600(8-5),1,일반,8,대,235.000,16,준주거지역,0,지정되지않음,...,2,평지,6,자루형,8,세로한면(가),1625000,0,지정되지않음,2021-05-07


In [116]:
pb_std_prc.columns

Index(['emd_jibun', '특수지구분코드', '특수지구분명', '지목코드', '지목', '토지면적', '용도지역코드1',
       '용도지역명1', '용도지구코드1', '용도지구명1', '저촉율', '토지이용상황코드', '토지이용상황', '지형높이코드',
       '지형높이', '지형형상코드', '지형형상', '도로측면코드', '도로측면', '공시지가', '소유형태코드', '소유형태명',
       '데이터기준일자'],
      dtype='object')

In [117]:
pb_std_prc.columns = ['emd_jibun','spc_grd_cd','spc_grd_info','jimok_cd','jimok_nm','lnd_area',
                      'mus_cd','mus_nm','dis_cd','dis_nm','con_rate','use_cd','use_nm','ht_cd','ht_nm',
                     'shp_cd','shp_nm','road_sd_cd','road_sd_nm','lnd_prc','pos_cd','pos_nm','date']

pb_std_prc.head(1)

,emd_jibun,spc_grd_cd,spc_grd_info,jimok_cd,jimok_nm,lnd_area,mus_cd,mus_nm,dis_cd,dis_nm,...,ht_cd,ht_nm,shp_cd,shp_nm,road_sd_cd,road_sd_nm,lnd_prc,pos_cd,pos_nm,date
0,2629010600(8-5),1,일반,8,대,235.000,16,준주거지역,0,지정되지않음,...,2,평지,6,자루형,8,세로한면(가),1625000,0,지정되지않음,2021-05-07


In [118]:
pb_std_prc.to_csv(input_path.joinpath('공공데이터_표준지공시지가.csv'), index_label=False)